In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from typing import Any

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [4]:
class MLP(nn.Module):
    """Our Multilayer Perceptron"""

    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(360, 90)
        self.fc2 = nn.Linear(90, 90)
        self.fc3 = nn.Linear(90, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [5]:
def load_data(train_csv: str, val_csv: str,
              test_csv: str) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor,
                                      torch.Tensor, torch.Tensor]:
    """
    :param train_csv:
    :param val_csv:
    :param test_csv:
    :return:
    """
    train_data = pd.read_csv(train_csv)
    val_data = pd.read_csv(val_csv)
    test_data = pd.read_csv(test_csv)

    # Take only features we need
    X_train = train_data.drop(['order0', 'order1', 'order2'], axis=1).values
    y_train = train_data['order0'].values

    X_val = val_data.drop(['order0', 'order1', 'order2'], axis=1).values
    y_val = val_data['order0'].values

    X_test = test_data.values

    # Normalization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.int64)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.int64)
    X_test = torch.tensor(X_test, dtype=torch.float32)

    return X_train, y_train, X_val, y_val, X_test

In [6]:
def init_model(lr: float) -> tuple[MLP, nn.modules.loss.CrossEntropyLoss,
                                   optim.Adam]:
    """
    :return:
    """
    model = MLP()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    return model, criterion, optimizer

In [7]:
def evaluate(model: MLP, X: pd.DataFrame,
             y: pd.Series) -> tuple[torch.Tensor, float, np.ndarray]:
    """
    :param model:
    :param X:
    :param y:
    :return:
    """
    model.eval()

    with torch.no_grad():
        predictions = model(X).argmax(dim=1)
        accuracy = accuracy_score(y, predictions)
        conf_matrix = confusion_matrix(y, predictions)

    return predictions, accuracy, conf_matrix

In [12]:
def train(model: MLP, criterion: nn.modules.loss.CrossEntropyLoss,
          optimizer: optim.Adam, X_train: torch.Tensor, y_train: torch.Tensor,
          X_val: torch.Tensor, y_val: torch.Tensor, epochs: int,
          batch_size: int):
    """
    :param model:
    :param criterion:
    :param optimizer:
    :param X_train:
    :param y_train:
    :param X_val:
    :param y_val:
    :param epochs:
    :param batch_size:
    :return:
    """
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for i in range(0, X_train.size(0), batch_size):
            optimizer.zero_grad()
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]

            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch.view(-1))
            loss.backward()
            optimizer.step()

            # print(f'\t Train: Epoch {epoch}, train Loss: {train_loss}')

            train_loss += loss.item()

        train_loss /= X_train.size(0)

        predictions, val_accuracy, conf_matrix = evaluate(model, X_val, y_val)
        print(f'Train: Epoch {epoch + 1}, val accuracy: {val_accuracy}')

    return model

In [9]:
def main(**kwargs: Any) -> None:
    """
    :param kwargs:
    """
    # Load data
    X_train, y_train, X_val, y_val, X_test = load_data(
        kwargs['train_csv'], kwargs['val_csv'], kwargs['test_csv'])

    # Initialize model
    model, criterion, optimizer = init_model(kwargs['lr'])

    # Train model
    model = train(
        model, criterion, optimizer,
        X_train, y_train, X_val, y_val,
        kwargs['num_epoches'], kwargs['batch_size']
    )

    # Predict on test set
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        predictions = outputs.argmax(dim=1)

    # dump predictions to 'submission.csv'
    pd.DataFrame(predictions).to_csv(kwargs['out_csv'])

In [14]:
train_csv = ('/content/drive/MyDrive/'
             'Проектирование по компьютерным технологиям в механике/'
             'hw1/data/train.csv')
val_csv = ('/content/drive/MyDrive/'
            'Проектирование по компьютерным технологиям в механике/'
            'hw1/data/val.csv')
test_csv = ('/content/drive/MyDrive/'
            'Проектирование по компьютерным технологиям в механике/'
            'hw1/data/test.csv')
out_csv = ('/content/drive/MyDrive/'
           'Проектирование по компьютерным технологиям в механике/'
           'hw1/submission.csv')
lr = 0.001
batch_size = 64
num_epoches = 15

main(train_csv=train_csv, val_csv=val_csv, test_csv=test_csv, out_csv=out_csv,
     lr=lr, batch_size=batch_size, num_epoches=num_epoches)

Train: Epoch 1, val accuracy: 0.754244749088165
Train: Epoch 2, val accuracy: 0.7727329895610615
Train: Epoch 3, val accuracy: 0.7902150672871336
Train: Epoch 4, val accuracy: 0.8141114325242108
Train: Epoch 5, val accuracy: 0.8203999496918627
Train: Epoch 6, val accuracy: 0.8293296440699283
Train: Epoch 7, val accuracy: 0.8439190038988806
Train: Epoch 8, val accuracy: 0.8492013583197082
Train: Epoch 9, val accuracy: 0.8445478556156458
Train: Epoch 10, val accuracy: 0.8412778266884668
Train: Epoch 11, val accuracy: 0.8520940762168281
Train: Epoch 12, val accuracy: 0.8607722299081877
Train: Epoch 13, val accuracy: 0.8547352534272419
Train: Epoch 14, val accuracy: 0.8566218085775374
Train: Epoch 15, val accuracy: 0.8536033203370645
